In [4]:
from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_provider import IBMProvider
import pandas as pd
import os
import json

import matplotlib.pyplot as plt
from datetime import datetime

import seaborn as sns

plt.rcParams.update({'font.size': 14, 'font.family': 'serif', 'mathtext.fontset': 'dejavuserif'})

In [5]:
provider = IBMProvider(token='ccb55572af16d8dbf7152a35cd3bbbaa2dfcd45fdc26efcc88b0e4673681c35253cbc04e1e9cfad7d484b71c3adc46f694abc774109e0acc36c5c129f2f48b38')  # used for feb       | l.boffa1@studenti.unisa.it: Google auth
#provider = IBMProvider(token='4f1de20d25e004d584cc5992ebe9bcd7de79dd3bbdb72805699ffbd9fcc6ef736041f46a30aa1a3542955945f27b542de74b035ff3991b503256f1ab77a36b6a') # used for mar 9:08s | latriade003@gmail.com: Google auth
#provider = IBMProvider(token='e4fba203211bb858b68aa8f8f717d390bb00ffcb5446f454b1447775b927c65d56d8beb2b90d7beb57078d8807f3013e9c0bcafb91215acfeca6ffa1f1e6f7a3') # used for feb 7:46s |  rofetob682@cashbn.com:Formula1&2&3
#provider = IBMProvider(token='ae0af2c735f631d165a0a0f4270fcf79bed032b916448b9345d2854cb124c7aa807e3e99674a758487949d3489f837fd658a37ac6361b4e53a27dba92482dfcc') # used for mar s     |  decer19500@confmin.com:Formula1&2&3 password errata
#provider = IBMProvider(token='718bb475e897f23387ea9298c0592bbbbee623bac0f5e3e04b2befdc1b33f119c4fe259c746f868c070a262391e7849340bdc33d891498024020d2074577804a') # used for mar 8:49s |  ketofes973@kytstore.com:Formula1&2&3
#provider = IBMProvider(token='5aa696df0acae89e3cf11ea6c406889353c2c06ae64466a445a3bac0ac89f81d2d449d3cd7cef89467ab77dbf1f23e850bb6108328084f9e800370408e1d7eb8') # used for mar 10:00s|  doyid36339@egvoo.com:Formula1&2&3

In [6]:
# Mostro i backend disponibili per l'esecuzione
print(provider.backends())

backend_maps = {
    index: backend.name.split('_')[1] for index, backend in enumerate(provider.backends())
}

# Visualizo che i nomi siano corretti 
print(backend_maps)

service = QiskitRuntimeService(channel='ibm_quantum',
    instance='ibm-q/open/main',
    token='ccb55572af16d8dbf7152a35cd3bbbaa2dfcd45fdc26efcc88b0e4673681c35253cbc04e1e9cfad7d484b71c3adc46f694abc774109e0acc36c5c129f2f48b38')
# IBM Quantum channel; set to default

# Inizializzo il backend array 
backend_array=[]
backend_map = backend_map = {index: backend.name.split('_')[1] for index, backend in enumerate(service.backends(operational=True, simulator=False))}

print("Macchine disponibili:")
for index, name in backend_map.items():
    print(f"{index}: {name}")
    
scelta = input("Scegli un'opzione (1/2/3): 1 meno occupata, 2 tutte, 3 scegli manualmente ")

# Esegui un'azione in base alla scelta
if scelta == "1":
    print("Hai scelto l'opzione 1!")
    print("Scelgo la macchina meno occupata")
    backend_array.append(service.least_busy(operational=True, simulator=False, min_num_qubits=127))
    backend_map = {
        index: backend.name.split('_')[1] for index, backend in enumerate(backend_array)
    }
    print(backend_array)
    print(backend_map)
elif scelta == "2":
    print("Hai scelto l'opzione 2!")
    for _, name in backend_maps.items():
            class_name = f"ibm_{name}"  
            try:
                backend_array.append(service.backend(class_name, instance='ibm-q/open/main'))
            except KeyError:
                print(f"Error: Class '{class_name}' not found in qiskit.test.mock.")
    backend_map = {
        index: backend.name.split('_')[1] for index, backend in enumerate(backend_array)
    }
    print(backend_array)
    print(backend_map)
elif scelta == "3":
    print("Hai scelto l'opzione 3!")
    scelte_utente = input("Inserisci gli indici delle macchine separate da una virgola: ")
    indici = [int(i.strip()) for i in scelte_utente.split(',') if i.strip().isdigit()]
    backend_array = []
    new_backend_map = {}
    for index in indici:
        if index in backend_map:
            class_name = f"ibm_{backend_map[index]}"
            try:
                backend = service.backend(class_name, instance='ibm-q/open/main')
                backend_array.append(backend)
                new_backend_map[len(new_backend_map)] = backend.name.split('_')[1]
            except Exception as e:
                print(f"Errore nel trovare la macchina '{class_name}': {e}")
    backend_map = new_backend_map
    print("Macchine selezionate:", [backend.name for backend in backend_array])
    print("Nuovo mapping aggiornato:", backend_map)
else:
    print("Scelta non valida!")

[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_kyiv')>]
{0: 'brisbane', 1: 'sherbrooke', 2: 'kyiv'}
Macchine disponibili:
0: kyiv
1: brisbane
2: sherbrooke
Hai scelto l'opzione 1!
Scelgo la macchina meno occupata
[<IBMBackend('ibm_sherbrooke')>]
{0: 'sherbrooke'}


In [7]:
def normalized_fractional_hamming_intradistance(dict1, dict2):
    # Verifica se i dizionari hanno le stesse chiavi
    if set(dict1.keys()) != set(dict2.keys()):
        raise ValueError("Response must have the same lenght")
    # Calcola la somma delle differenze assolute tra i valori corrispondenti
    sum_diff = sum(abs((dict1[key] - dict2[key])) for key in dict1.keys())
    fractional_hamming_distance = sum_diff / 2 #len(dict1) perchè la somma ha come range [0,2] quindi /2 per normalizzare a range [0,1]
    return fractional_hamming_distance

In [8]:
import pickle
n_challenge = 10
n_qubits = 8
shots= 1000
n_runs = 5 # minimun 2 runs altrimenti instability non viene calcolata per come è stata pensata

# Loading from file (this is the only one file used for all the metrics)
file_name = f'./results/configuration_challenges/random_arrays_{n_challenge}.pkl'

if os.path.exists(file_name):
    with open(file_name, 'rb') as f:
        a_challenge, b_challenge, c_challenge, d_challenge, e_challenge, f_challenge = pickle.load(f)
    print(f"Arrays loaded from '{file_name}'")
else:
    print(f"Error: File '{file_name}' not found.")

Arrays loaded from './results/configuration_challenges/random_arrays_10.pkl'


In [9]:
import numpy as np

def puf_quantum_circuit(i,num_qubits=8, depth=4, save_dir="./results/circuit"):
    qc = QuantumCircuit(num_qubits, num_qubits)  # Include classical bits for measurement
    
    for _ in range(depth):
        for qubit in range(num_qubits):
            gate_choice = np.random.choice(['h', 'x', 'rx', 'rz'])
            if gate_choice in ['h', 'x']:
                getattr(qc, gate_choice)(qubit)
            elif gate_choice == 'rx':
                angle = angle = a_challenge[i][qubit]
                qc.rx(angle, qubit)
            elif gate_choice == 'rz':
                angle = b_challenge[i][qubit]
                qc.rz(angle, qubit)
        
        # Random CNOT gates to introduce entanglement
        for _ in range(num_qubits // 2):
            control, target = np.random.choice(num_qubits, 2, replace=False)
            qc.cx(control, target)
    
    # Add measurement
    qc.measure(range(num_qubits), range(num_qubits))
    
    # Creazione della cartella se non esiste
    os.makedirs(save_dir, exist_ok=True)
    
    # Determina il numero progressivo del file
    existing_files = [f for f in os.listdir(save_dir) if f.startswith("circuit_") and f.endswith(".pkl")]
    circuit_number = len(existing_files) + 1
    save_path = os.path.join(save_dir, f"circuit_{circuit_number}.pkl")
    
    # Salva il circuito con numero progressivo
    with open(save_path, "wb") as f:
        pickle.dump(qc, f)
    
    return qc, save_path

def load_quantum_circuit(circuit_number, save_dir="quantum_circuits"):
    """Carica un circuito specifico dato il numero."""
    file_path = os.path.join(save_dir, f"circuit_{circuit_number}.pkl")
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            return pickle.load(f)
    else:
        raise FileNotFoundError(f"Circuito {circuit_number} non trovato.")


In [10]:
def qpuf_ibm(qc,backend):
  
  pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
  isa_circuit = pm.run(qc)
  
  sampler = Sampler(backend)
  job = sampler.run(isa_circuit, shots=shots)
  result = job.result() 

  pub_result = []

  for i in range(n_challenge*n_runs):
    pub_result.append(result[i].data.c.get_counts())
    # da controllare con franco 
    #print(f" >> c output register counts: {result[i].data.c.get_counts()}")
  
  #lista delle responses formattate
  responses=[]
  for j in range(len(pub_result)):
    #iesima response
    response = {}
    for i in range(2**n_qubits):
      binary_string = format(i, '0' + str(n_qubits) + 'b')
      #se trova la combinazione copia il valore e lo divide per il numero dei shots per normalizzarlo tra 0 e 1, altrimenti mette 0
      response[binary_string] = pub_result[j].get(binary_string,0)/shots
    responses.append(response)
  return responses

def run_qcirc(i):
  qc=[]
  for n in range(n_challenge):
      for _ in range(n_runs):
          #metti in fila tutti i circuiti
          qc.append(puf_quantum_circuit(i=n))
  r=qpuf_ibm(qc,backend_array[i])

  #cosi salvo il backend e utilizzo lo stesso per le altre metriche 
  file_path = f'./results/backend/real/backend_{backend_map[i]}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.json'
  if not os.path.exists(file_path):
    with open(file_path, 'w') as json_file:
        json.dump(r, json_file)
    print(f"Backend_{backend_map[i]} salvato correttamente")
  else:
    print(f"Il file {file_path} esiste già. Salvataggio annullato.")

  
def instability(i):
  inst_table = pd.DataFrame(columns=["n-th challenge", "i-th execution", "j-th execution", "dist"])
    
  file_path = f'./results/backend/real/backend_{backend_map[i]}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.json'
  with open(file_path, 'r') as json_file:
    response=json.load(json_file)

  for n in range(n_challenge):
      #estrae i circuiti solo per quella challenge
      same_response = [response[n*n_runs + k] for k in range(n_runs)]

      for i in range(len(same_response)):
          for j in range(i + 1, len(same_response)):
              distance = normalized_fractional_hamming_intradistance(same_response[i], same_response[j])
              inst_table.loc[len(inst_table)] = [n, i, j, distance]
    
  inst_table[['n-th challenge', 'i-th execution', 'j-th execution']] = inst_table[['n-th challenge', 'i-th execution', 'j-th execution']].astype(int)
  return inst_table

def randomness(i): 
  rand_table =pd.DataFrame(columns=["n-th challenge", "m-th challenge", "dist"])
  #cosi mi ricrea il backend e perdo il salvataggio precendente
 
  file_path = f'./results/backend/real/backend_{backend_map[i]}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.json'
  with open(file_path, 'r') as json_file:
    response=json.load(json_file)

  all_same_runs= []

  for i in range(n_challenge):
    same_runs = response[0+(i*n_runs)]  #devo skippare nruns alla volta e le isolo prima (cioe le tiro fuori)
    all_same_runs.append(same_runs)
  
  for n in range(len(all_same_runs)):
    for m in range(n + 1, len(all_same_runs)): # per evitare di calcolare la distanza tra le stesse coppie ma ordine invertito
      distance = normalized_fractional_hamming_intradistance(all_same_runs[n], all_same_runs[m])
      rand_table.loc[len(rand_table)] = [n, m, distance]
    
  rand_table['n-th challenge']=rand_table['n-th challenge'].astype(int)
  rand_table['m-th challenge']=rand_table['m-th challenge'].astype(int)
  return rand_table

def uniqueness():
  unique_table = pd.DataFrame(columns=["i-th device","j-th device","n-th challenge", "dist"])
  responses=[]
  for i in range(len(backend_array)):
    file_path = f'./results/backend/real/backend_{backend_map[i]}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.json'
    with open(file_path, 'r') as json_file:
      r=json.load(json_file)
    responses.append(r)  
  
  # Mantengo la struttura: macchina -> challenge -> run
  all_same_runs = []
  for i in range(len(backend_array)):  # Lunghezza dei backend
      machine_challenges = []  # Lista per le challenge di questa macchina
      for j in range(n_challenge):
          # Estrai i run per questa challenge
          same_runs = responses[i][j*n_runs]  # Prendo tutti i challenge per della stessa runs
          machine_challenges.append(same_runs)  # Aggiungo le challenge della run alla lista della macchina
      all_same_runs.append(machine_challenges)  # Aggiungo la struttura della macchina
  
  for i in range(len(backend_array)): #lunghezza backend
    for j in range(i + 1, len(backend_array)):
      for n in range(n_challenge):
        #print(f"macchina {i} e macchina {j} su challenge {n}")
        distance = normalized_fractional_hamming_intradistance(all_same_runs[i][n], all_same_runs[j][n])
        unique_table.loc[len(unique_table)] = [i,j,n, distance]
  unique_table['i-th device']=unique_table['i-th device'].astype(int)
  unique_table['j-th device']=unique_table['j-th device'].astype(int)
  unique_table['n-th challenge']=unique_table['n-th challenge'].astype(int)
  return unique_table

for i in range(len(backend_array)):
  print(backend_array)
  print(backend_map)
  run_qcirc(i)

[<IBMBackend('ibm_sherbrooke')>]
{0: 'sherbrooke'}


AttributeError: 'tuple' object has no attribute '_data'

In [ ]:
if len(backend_array) < 1:
    raise ValueError("Errore: Nessuna macchina selezionata, impossibile eseguire i test.")

current_time = datetime.now().strftime("%Y-%m-%d__%H-%M")
folder_name = f'nq{n_qubits}_nc{n_challenge}_s{shots}_{n_runs}_{current_time}'
# Percorso della cartella da creare (puoi personalizzarlo)
folder_path_i = f'./results/instability/real/{folder_name}'
folder_path_r = f'./results/randomness/real/{folder_name}'
folder_path_u = f'./results/uniqueness/real/{folder_name}'

# Controlla se la cartella esiste già, altrimenti la crea
for folder_path in [folder_path_i, folder_path_r, folder_path_u]:
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Cartella '{folder_path}' creata con successo.")
    else:
        print(f"La cartella '{folder_path}' esiste già.")

inst_table=[]

rand_table=[]

for i, backend_name in backend_map.items():
    if(n_runs < 2):
       print("Instability non calcolabile con una sola runs!")
    else:
      inst_table.append(instability(i))
      file_path = f'./results/instability/real/{folder_name}/inst_table_{backend_name}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.csv'
      inst_table[i].to_csv(file_path, index=False)
      print("Dati salvati correttamente in:", file_path)

    rand_table.append(randomness(i))
    file_path = f'./results/randomness/real/{folder_name}/rand_table_{backend_name}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.csv'
    rand_table[i].to_csv(file_path, index=False)
    print("Dati salvati correttamente in:", file_path)

if(len(backend_array) < 2):
  print("Uniqueness non calcolabile con una sola macchina!")
else:
  unique_table=uniqueness()
  file_path = f'./results/uniqueness/real/{folder_name}/unique_table_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.csv'
  unique_table.to_csv(file_path, index=False)
  print("Dati salvati correttamente in:", file_path)
  unique_table
    
inst_table[0]
rand_table[0]

In [ ]:
if(n_runs < 2):
    print("Instability non calcolabile con una sola runs!")
else:
    for i, (backend_name, tab) in enumerate(zip(backend_map.values(), inst_table)):
        sns.kdeplot(tab['dist'])
        # Setto il titolo del grafico alla relativa macchina
        plt.title(f'Instability Real - {backend_name.capitalize()}')
        plt.gca().spines['top'].set_visible(False)
        plt.gca().spines['right'].set_visible(False)
        plt.xlabel('Distance')
        # Salvo con il nome relativo della macchina
        plt.savefig(f'./results/instability/real/{folder_name}/instability_{backend_name}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.png')
        # Visualizzo il grafico
        plt.show()

In [ ]:
for i, (backend_name, tab) in enumerate(zip(backend_map.values(), rand_table)):
    sns.kdeplot(tab['dist'])
    plt.title(f'Randomness Real - {backend_name.capitalize()}')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xlabel('Distance')
    
    plt.savefig(f'./results/randomness/real/{folder_name}/randomness_real_{backend_name}_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.png')
    
    plt.show()

In [ ]:
if(len(backend_array) < 2):
    print("Uniqueness non calcolabile con meno di 2 macchine")
else:
    sns.kdeplot(unique_table['dist'])
    plt.title('Uniqueness Real')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    plt.xlabel('Distance')

    plt.savefig(f'./results/uniqueness/real/{folder_name}/uniqueness_nq{n_qubits}_nc{n_challenge}_s{shots}_nr{n_runs}.png')

    plt.show()